In [ ]:
# Import necessary libraries
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
# may have to turn this into a script 

In [ ]:

# X data
fp = '/rc_scratch/reba1583/variable_yr_files_4/'

IWV = xr.open_mfdataset(fp+'IWV').IWV.values
EFLUX = xr.open_mfdataset(fp+'EFLUX').EFLUX.values
LWTNET = xr.open_mfdataset(fp+'LWTNET').LWTNET.values
SF = xr.open_mfdataset(fp+'SF').sf.values
SLP = xr.open_mfdataset(fp+'SLP').SLP.values
T = xr.open_mfdataset(fp+'T').T.values
U = xr.open_mfdataset(fp+'U').U.values
V = xr.open_mfdataset(fp+'V').V.values

In [ ]:
data = np.stack([ IWV, EFLUX, LWTNET,
                 SF, SLP, T, U, V])



In [ ]:
del V 
del U
del T
del IWV
del EFLUX
del SF
del SLP
del LWTNET

In [ ]:
# Y data
Y = xr.open_mfdataset('/rc_scratch/reba1583/Y_data/Y_AR_only.nc')
Y = Y.Y.values

In [ ]:
# times for final xarray
variable_times = xr.open_mfdataset(fp+'U').time

In [ ]:
var_data = dict(
    features = (['n_channel', 'time', 'lat', 'lon' ], data),
    labels_2d = (['time', 'lat', 'lon'], Y)
)

coords = dict(
    n_channel = (['n_channel'], np.array(['IWV', 'EFLUX', 'LWTNET', 'SF', 'SLP', 'T', 'U', 'V'])),
    time = (['time'], pd.to_datetime(np.array(variable_times))),
    
)


In [ ]:
ds = xr.Dataset(
    data_vars = var_data, 
    coords = coords
)


In [ ]:
# check shape here - compare to 4day testing I think you need to rearrange them

In [ ]:
## get even number of timesteps with/without ARs
AR_index = np.squeeze(np.where(ds.Y.max(dim = ('lat','lon')).load()==1))
noAR_index = np.setdiff1d(np.arange(len(ds.time)), AR_index)
np.random.shuffle(noAR_index)
noAR_index = noAR_index[0:len(AR_index)]
select = np.sort(np.concatenate((noAR_index, AR_index)))
data = ds.isel(time = select)




In [ ]:
# split into training, validating, and testing
index = np.arange(len(y_data.time))
split1, split2 = int(.7*len(index)), int(.85*len(index))

np.random.shuffle(index)
index_train, index_validate, index_test = index[:split1], index[split1:split2], index[split2:]
index_train.sort()
index_validate.sort()
index_test.sort()

ds_train = data.isel(time = index_train)
ds_test = data.isel(time = index_test)
ds_validate = data.isel(time = index_validate)



In [ ]:
ds_train.to_netcdf('rc_scratch/reba1583/train')
ds_test.to_netcdf('rc_scratch/reba1583/test')
ds_validate.to_netcdf('rc_scratch/reba1583/validate')

